In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir("../")

In [3]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [14]:
from pathlib import Path
from dataclasses import dataclass
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    params_image_size:list
    params_batch_size:int

In [22]:
from deepclassifier.constants import *
from deepclassifier.utils import read_yaml,create_directories,save_json

In [23]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config.evaluation

        evaluation_config = EvaluationConfig(
            path_of_model=config.path_of_model,
            training_data=config.training_data,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE 
        )

        return evaluation_config

In [29]:
import os
#import urllib.request as request
#from zipfile import ZipFile
import tensorflow as tf
import time
from pathlib import Path
#from deepclassifier.entity import EvaluationConfig
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
       model = self.load_model(self.config.path_of_model)
       self._valid_generator()
       self.score = model.evaluate(self.valid_generator)
       
    def save_score(self):
        scores = {"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

In [30]:
try:
    config = ConfigurationManager()
    data_evaluation_config = config.get_evaluation_config()
    data_evaluation = Evaluation(config=data_evaluation_config)
    data_evaluation.evaluation()
    data_evaluation.save_score()
except Exception as e:
    raise e

Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1042s 2s/step - loss: 1.1268 - accuracy: 0.8926
